In [31]:
import pandas as pd
from jaal import Jaal
from jaal.datasets import load_got

In [32]:
sz_file_path = "./dump.csv"
sz_file_csv = pd.read_csv(sz_file_path,usecols=['initiatingDeviceName','receivingDeviceName', 'connectionCount'],
                          header=0,
                       
                        )

In [33]:
df = sz_file_csv.rename(
    columns={
        "initiatingDeviceName": "from",
        "receivingDeviceName": "to",
        "connectionCount": "weight"
    }
)

In [34]:
df_grouped = df.groupby(['from','to'], as_index=False)['weight'].sum()

In [35]:
df = sz_file_csv.rename(
    columns={
        "initiatingDeviceName": "id",
        "receivingDeviceName": "to",
        "connectionCount": "weight",

    }
)
df_nodes = df.groupby(['id'], as_index=False)['weight'].sum()

In [36]:
Jaal(df_grouped).plot(port=8071)

Parsing the data...Done


No trigger
inside color node title
Modifying edge size using  weight
